# Grid Oscillations

With this notebook, you can experiment with grid oscillations connected to nullspace consistent classical finite difference summation by parts operators. See Section 3.1 of the paper.

Run the cell below to activate the project environment containing the necessary packages.

In [ ]:
]activate .

In [ ]:
using SummationByPartsOperators
using LinearAlgebra, SparseArrays
using Printf
using Plots, LaTeXStrings
using Interact

function matrices(N, order)
    d_op = derivative_operator(MattssonNordström2004(), 1, order, 0, 1//1, N)
    d = Matrix(d_op)
    m = mass_matrix(d_op)
    id = one(m)
    
    d, m, id
end

function grid_oscillations(N, order)
    D, M, Id = matrices(N, order)
    osc = nullspace(M \ (D' * M))
    osc ./ abs(osc[N ÷ 2])
end

function plot_grid_oscillations(N, order)
    osc = grid_oscillations(N, order)
    scatter(osc, xguide="Node", yguide="osc", label="")
end


@manipulate for N in 3:100, order in [2, 4, 6, 8]
    plot_grid_oscillations(N, order)
end

## Create plots used in the paper

In [ ]:
import PyPlot; plt = PyPlot
using PyCall

# line cycler adapted to colourblind people
cycler = pyimport("cycler")
cycler.cycler
colours = ["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]
linestyles = ["-", "--", "-.", ":", "-", "--", "-."]
#markers = [">", "^", "<", "v", "+", "x", "."]
markers = ["4", "2", "3", "1", "+", "x", "."]
colourblind_cycler = (cycler.cycler(color=colours) + cycler.cycler(linestyle=linestyles))
plt.rc("axes", prop_cycle=colourblind_cycler)

plt.rc("text", usetex=true)
plt.rc("text.latex", preamble="\\usepackage{newpxtext}\\usepackage{newpxmath}\\usepackage{commath}\\usepackage{mathtools}")
plt.rc("font", family="serif", size=18.)
plt.rc("savefig", dpi=200)
plt.rc("legend", fontsize="medium", fancybox=true, framealpha=0.5)


function create_plots(N)
    isdir("../figures") || mkdir("../figures")
    filename = joinpath("../figures", "grid_oscillations_N_$N.pdf")
    
    plt.clf()
    x = range(0., stop=1., length=N)
    for (iter,order) in enumerate(2:2:8)
        plt.scatter(x, grid_oscillations(N, order), label="Order $order", marker=markers[iter], s=100, lw=2)
    end
    #plt.xlim(extrema(x))
    plt.xlabel(L"x")
    plt.ylabel(L"\mathrm{osc}")
    plt.legend(loc="best", ncol=2)
    plt.savefig(filename, bbox_inches="tight")
end


create_plots(50)
create_plots(51)